In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from abc import abstractmethod

In [2]:
OUTPUT_PATH = 'output/'
INPUT_PATH = 'dataset/'
BASE_FILE_NAME='btcusdt'
# TIME_FRAMES=['1h','3m','5m','15m','30m']
TIME_FRAMES=['1h']

SIGNALS = [-1,0,1]
PRICE_TYPE = ['open','close','high','low']

In [3]:
class Trade:
    
    def __init__(self) -> None:
        self.timeStamp = None
        self.signal = None
        self.price_type = None
    
    def execute(self,timeStamp:datetime ,signal:int,price_type:str) -> object:
        
        assert signal in SIGNALS, f"Invalid Signal {signal}"
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"

        self.timeStamp = timeStamp
        self.signal = signal
        self.price_type = price_type
        return self
    
    def __str__(self) -> str:
        return f"{self.timeStamp} {self.signal} {self.price_type}"
    
class PrVO:

    def __init__(self,open:float,high:float,low:float,close:float,volume:int,timeStamp:datetime) -> None:
        self.open = open
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume
        self.timeStamp = timeStamp
        self.returns = (close - open)/open
        self.candle_color = 'GREEN' if self.close > self.open else 'RED'

    def __repr__(self) -> str:
        return f"Price-Volume : open={self.open}, high={self.high},low={self.low}, close={self.close}, volume={self.volume}, timeStamp={self.timeStamp}"
    
    def __str__(self) -> str:
        return f"Price-Volume : open={self.open}, high={self.high},low={self.low}, close={self.close}, volume={self.volume}, timeStamp={self.timeStamp}"

In [4]:
class TradingTerminal:

    def __init__(self) -> None:
        self.time=None
        self.PrVo=dict()
        self.cash_balance=1000
        self.tradeHistory=[]
        
        for timeframe in TIME_FRAMES:
            self.readData(BASE_FILE_NAME+'_'+timeframe+'.xlsx')

    def readData(self,filepath):
        df=pd.read_excel(INPUT_PATH+filepath)
        
        data=[]
        for index,row in df.iterrows():
            data.append(PrVO(row['open'],row['high'],row['low'],row['close'],row['volume'],row['datetime']))

        data_timeframe=filepath.split('/')[-1].split('.')[0].split('_')[1]
        self.PrVo[data_timeframe]=np.array(data)
        
    def takeTrade(self,timeStamp,signal,price_type):
        assert signal in SIGNALS, f"Invalid Signal {signal}"
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"

        trade=Trade().execute(timeStamp,signal,price_type)
        self.tradeHistory.append(trade)

    def outputTradeHistory(self):
        file=open(OUTPUT_PATH+'trade_history.txt','w')
        for trade in self.tradeHistory:
            file.write(str(trade)+'\n')

    def setTime(self,time):
        self.time=time

    def getSimpleMovingAverage(self,lag,timeframe,price_type):
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"
        assert timeframe in TIME_FRAMES, f"Invalid Time Frame {timeframe}"
        
        prices = np.array([getattr(entry, price_type) for entry in self.PrVo[timeframe]])
        sma = np.convolve(prices, np.ones(lag) / lag, mode='valid')
        sma = np.concatenate([np.full(lag - 1, np.nan), sma])
        return sma
    


In [5]:
class Strategy:
    def __init__(self,TradingTerminal) -> None:
        self.terminal=TradingTerminal

    @abstractmethod
    def run(self, *args):
        pass

In [6]:
class Strategy1(Strategy):

    def __init__(self,TradingTerminal) -> None:
        super().__init__(TradingTerminal)

    def run(self,lag1,lag2,price_type,timeframe):
        assert price_type in PRICE_TYPE, f"Invalid Price Type {price_type}"
        assert lag1>lag2, f"Invalid Lag {lag1} {lag2}"
        assert timeframe in self.terminal.PrVo.keys(), f"Invalid Timeframe {timeframe}"

        PrVo=self.terminal.PrVo[timeframe]

        ema1=self.terminal.getSimpleMovingAverage(lag1,timeframe,price_type)
        ema2=self.terminal.getSimpleMovingAverage(lag2,timeframe,price_type)

In [7]:
tt=TradingTerminal()

In [8]:
s1=Strategy1(tt)

In [9]:
s1.run(10,5,'open','1h')